In [1]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Add Lincoln to system path
import sys
sys.path.append("/Users/seth/development/lincoln/")

In [3]:
# %load_ext autoreload
# %autoreload 2

In [25]:
import torch
from torch import Tensor

from lincoln.layers import Dense
from lincoln.losses import LogSoftmaxLoss, LogSigmoidLoss, MeanSquaredError
from lincoln.optimizers import SGD
from lincoln.operations.activations import Sigmoid, LogSigmoid
from lincoln.network import NeuralNetwork

## Regression network

In [26]:
from sklearn.datasets import load_boston

boston = load_boston()

data = boston.data
target = boston.target
features = boston.feature_names

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)

## Regression models

In [27]:
from lincoln.losses import MeanSquaredError
from lincoln.optimizers import SGD

from lincoln.operations.activations import Sigmoid, LinearAct

#### Linear Regression

In [28]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError(),
            optimizer = SGD())

lr.fit(X_train, y_train, X_test, y_test,  
       epochs = 10,
       eval_every = 1,
       seed=82618,
       single_output=True);

Validation loss after 1 epochs is 4404.868
Validation loss after 2 epochs is 3859.566
Validation loss after 3 epochs is 3753.522
Validation loss after 4 epochs is 3802.207
Validation loss after 5 epochs is 3977.285
Validation loss after 6 epochs is 3975.196
Validation loss after 7 epochs is 4758.660
Validation loss after 8 epochs is 4110.424
Validation loss after 9 epochs is 3970.611
Validation loss after 10 epochs is 4082.891


In [29]:
def mae(y_true, y_pred):
    return round(torch.mean(torch.abs(y_true - y_pred)).item(), 4)

def mse(y_true, y_pred):
    return round(torch.mean(torch.pow(y_true - y_pred, 2)).item(), 4)

In [30]:
def eval_regression_model(lr: NeuralNetwork, 
                          X_test: Tensor, 
                          y_test: Tensor):
    preds = lr.forward(X_test)
    preds = preds.reshape(preds.shape[0])
    return mae(preds, y_test), mse(preds, y_test)

In [31]:
eval_regression_model(lr, X_test, y_test)

(3.6518, 26.8611)

Works!

#### "Neural Network" Regression

In [32]:
nn = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError(),
            optimizer = SGD())

nn.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
           single_output=True);

Validation loss after 10 epochs is 2421.626
Validation loss after 20 epochs is 1583.616
Validation loss after 30 epochs is 1485.388
Validation loss after 40 epochs is 1567.009
Validation loss after 50 epochs is 1532.060
Validation loss after 60 epochs is 1729.032
Validation loss after 70 epochs is 1768.350
Validation loss after 80 epochs is 1659.488
Validation loss after 90 epochs is 1612.138
Validation loss after 100 epochs is 1579.115


In [33]:
eval_regression_model(nn, X_test, y_test)

(2.2825, 10.3889)

Works!

#### "Deep Learning" Regression

In [34]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=LinearAct())], 
            loss = MeanSquaredError(),
            optimizer = SGD())

dl.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
           single_output=True);

Validation loss after 10 epochs is 2225.604
Validation loss after 20 epochs is 2655.737
Validation loss after 30 epochs is 1803.951
Validation loss after 40 epochs is 1675.552
Validation loss after 50 epochs is 1384.854
Validation loss after 60 epochs is 1790.974
Validation loss after 70 epochs is 1282.833
Validation loss after 80 epochs is 1245.037
Validation loss after 90 epochs is 1302.036
Validation loss after 100 epochs is 1333.448


In [35]:
eval_regression_model(dl, X_test, y_test)

(2.0576, 8.7727)

Works!

## Classification models

#### Data prep (SciKit Learn)

In [36]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [37]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=82618)

X_train, X_test, y_train, y_test = Tensor(X_train), Tensor(X_test), Tensor(y_train), Tensor(y_test)

#### "`LogSigmoid`" Logistic Regression

In [38]:
logr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=LogSigmoid())],
            loss = LogSigmoidLoss(),
            optimizer = SGD())

logr.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
           single_output=True);

Validation loss after 10 epochs is 23.003
Validation loss after 20 epochs is 19.235
Validation loss after 30 epochs is 18.084
Validation loss after 40 epochs is 17.586
Validation loss after 50 epochs is 17.340
Validation loss after 60 epochs is 17.210
Validation loss after 70 epochs is 17.108
Validation loss after 80 epochs is 17.061
Validation loss after 90 epochs is 17.031
Validation loss after 100 epochs is 17.019


In [39]:
logr_mse = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
            optimizer = SGD())

logr_mse.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=82618,
           single_output=True);

Validation loss after 10 epochs is 10.025
Validation loss after 20 epochs is 7.280
Validation loss after 30 epochs is 6.160
Validation loss after 40 epochs is 5.527
Validation loss after 50 epochs is 5.118
Validation loss after 60 epochs is 4.818
Validation loss after 70 epochs is 4.576
Validation loss after 80 epochs is 4.395
Validation loss after 90 epochs is 4.274
Validation loss after 100 epochs is 4.190


In [40]:
def accuracy(y_true, y_pred):
    return round(torch.sum(torch.eq(y_true, y_pred)).item() / y_true.size()[0], 4)

In [41]:
def eval_classification_model(model: NeuralNetwork, 
                              X_test: Tensor, 
                              y_test: Tensor,
                              log_probs: bool = False):
    preds = model.forward(X_test)
    if log_probs:
        preds = torch.exp(preds)
    preds = preds > 0.5
    preds = preds.reshape(preds.shape[0]).type(torch.FloatTensor)  
    return accuracy(preds, y_test)

In [42]:
eval_classification_model(logr, X_test, y_test, log_probs=True)

0.9766

In [43]:
eval_classification_model(logr_mse, X_test, y_test)

0.9708

`LogSoftmaxLoss` works!

#### "Neural Network" Logistic Regression

In [44]:
logr_nn = NeuralNetwork(
    layers=[Dense(neurons=30, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
            optimizer = SGD())

logr_nn.fit(X_train, y_train, X_test, y_test,  
       epochs = 500,
       eval_every = 50,
       seed=82618,
           single_output=True);

Validation loss after 50 epochs is 7.861
Validation loss after 100 epochs is 6.662
Validation loss after 150 epochs is 5.946
Validation loss after 200 epochs is 5.438
Validation loss after 250 epochs is 5.053
Validation loss after 300 epochs is 4.753
Validation loss after 350 epochs is 4.513
Validation loss after 400 epochs is 4.322
Validation loss after 450 epochs is 4.171
Validation loss after 500 epochs is 4.052


In [45]:
eval_classification_model(logr_nn, X_test, y_test)

0.9766

**Works!**

## Convolution Demos

**Note:** Old losses and layers code.

In [47]:
from lincoln.layers import Conv2D
from lincoln.operations.activations import ReLU

In [48]:
from torchvision.datasets import MNIST
mnist_trainset = MNIST(root="./explanatory/data", train=True, download=False, transform=None)
mnist_testset = MNIST(root="./explanatory/data", train=False, download=True, transform=None)

#### One hot encode labels

In [49]:
data = mnist_trainset
num_labels = len(data.train_labels)
train_labels = torch.zeros(num_labels, 10)
for i in range(num_labels):
    train_labels[i][data.train_labels[i]] = 1
train_labels.shape

torch.Size([60000, 10])

In [50]:
data = mnist_testset
num_labels = len(data.test_labels)
test_labels = torch.zeros(num_labels, 10)
for i in range(num_labels):
    test_labels[i][data.test_labels[i]] = 1
test_labels.shape

torch.Size([10000, 10])

In [51]:
mnist_train = mnist_trainset.train_data.type(torch.float32).unsqueeze(3) / 255.0
mnist_test = mnist_testset.test_data.type(torch.float32).unsqueeze(3) / 255.0

In [52]:
X_train = mnist_train
X_test = mnist_test
y_train = train_labels
y_test = test_labels

In [53]:
def _conv_transform(input_: Tensor) -> Tensor:

    return input_.view(input_.shape[0],
                       input_.shape[1] * input_.shape[2] * input_.shape[3])

X_train_flat = _conv_transform(X_train)
X_test_flat = _conv_transform(X_test)

In [54]:
conv_nn_ll = NeuralNetwork(
    layers=[Dense(neurons=250, 
                  activation=Sigmoid()),
            Dense(neurons=50, 
                  activation=Sigmoid()),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [55]:
import time
start = time.time()
conv_nn_ll.fit(X_train_flat, y_train, X_test_flat, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=100,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

Validation loss after 1 epochs is 23464.492
Training took 3.4 seconds.


In [56]:
def eval_nn(nn, X_test, y_test):
    test_preds = nn.forward(X_test)
    
    test_preds_l = torch.argmax(test_preds, dim=1)
    
    test_vals_l = torch.argmax(y_test, dim=1)
    
    return torch.sum(test_preds_l == test_vals_l)

In [57]:
eval_nn(conv_nn_ll, X_test_flat, y_test)

tensor(9311)

In [58]:
X_train_perm = X_train.permute(0, 3, 1, 2)
X_test_perm = X_test.permute(0, 3, 1, 2)

In [59]:
# settings
pytorch = True

In [ ]:
conv_nn_logloss = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   pytorch=pytorch),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   pytorch=pytorch,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_logloss.fit(X_train_perm, y_train, X_test_perm, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=100,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

In [ ]:
eval_nn(conv_nn_logloss, X_test_perm, y_test)

### Testing network without PyTorch layers 

In [ ]:
# %load_ext autoreload
# %autoreload 2

#### No Cython

In [ ]:
pytorch = False
cython = False

In [ ]:
conv_nn_normal = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_normal.fit(X_train_perm, y_train, X_test_perm, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=1,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")

#### Cython

In [ ]:
pytorch = False
cython = True

In [ ]:
conv_nn_cython = NeuralNetwork(
    layers=[Conv2D(param_size=3,
                   out_channels=24,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython),
            Conv2D(param_size=3,
                   out_channels=30,
                   activation=ReLU(),
                   pytorch=pytorch,
                   cython=cython,
                   flatten=True),
            Dense(neurons=10, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(),
            optimizer = SGD(0.1))

In [ ]:
import time
start = time.time()
conv_nn_cython.fit(X_train_perm, y_train, X_test_perm, y_test,  
        epochs=1,
        eval_every=1,
        batch_size=1,
        seed=91418);
print("Training took", round(time.time()-start, 1), "seconds.")